# vid2e event generator

In order to use this code, we need a new conda environment.

In [ ]:
import esim_py
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
def viz_events(events, resolution):
    pos_events = events[events[:,-1]==1]
    neg_events = events[events[:,-1]==-1]

    image_pos = np.zeros(resolution[0]*resolution[1], dtype="uint8")
    image_neg = np.zeros(resolution[0]*resolution[1], dtype="uint8")

    np.add.at(image_pos, (pos_events[:,0]+pos_events[:,1]*resolution[1]).astype("int32"), pos_events[:,-1]**2)
    np.add.at(image_neg, (neg_events[:,0]+neg_events[:,1]*resolution[1]).astype("int32"), neg_events[:,-1]**2)

    image_rgb = np.stack(
        [
            image_pos.reshape(resolution),
            image_neg.reshape(resolution),
            np.zeros(resolution, dtype="uint8")
        ], -1
    )

    return image_rgb

## Convert video to event data

Make sure your video is scaled to a rather low resolution, e.g.
`ffmpeg -i input.avi -vf scale=320:240 output.avi`

### preprocessing

Put your video file into the folder `input` and run the following command.
In the upsampling folder, run `python upsample.py --input_dir=input --output_dir=output`

If you have a GPU, you can add `--device=cuda:0` to speed up the process, otherwise it can take a little while (try with a short video for the beginning). The frame images and timestamp file will be in the output directory.

### frames to events

Now you can run the esim code to simulate events from the video frame images.

In [ ]:
Cp, Cn            = 0.1, 0.1
refractory_period = 1e-4
log_eps           = 1e-3
use_log           = True

input_directory = "/home/user/output/"
image_folder    = input_directory + "imgs/"
timestamps_file = input_directory + "timestamps.txt"

In [ ]:
esim = esim_py.EventSimulator(Cp, Cn, refractory_period, log_eps, use_log)

esim.setParameters(Cp, Cn, refractory_period, log_eps, use_log)

events = esim.generateFromFolder(image_folder, timestamps_file)

print(events.shape)

Now we have our event data, ready for analysis! We have the events in a (N,4)-array, where the 4 coordinates are (x,y,t,+1/-1), giving us up or down events.

### Visualize the events in a new video

To visualise the events, we can make a new video, by aggregating the events that happened between the frames.

In [ ]:
H, W             = 256, 240
out_fps          = 30
end_time         = events[-1,2]
output_directory = "/home/mirieder/event_video/"
pixel_augment    = 150

def makeFrame(events, start, end, fname):
    events1 = events[events[:,2] >= start]
    events2 = events1[events1[:,2] < end]
    
    image_rgb = viz_events(events2, [H, W]) * pixel_augment
    plt.imsave(fname, image_rgb)

    return image_rgb

num_frames = int(end_time*out_fps) + 1
delta_t    = 1/out_fps

for k in range(num_frames):
    t_start = k*delta_t
    makeFrame(events, t_start, t_start+delta_t, output_directory + "frames/{0:04d}.png".format(k))

### FFMPEG command

Run the following command in a different terminal to convert the frame images to a video file (adjust the file paths):
`ffmpeg -y -framerate 30 -pattern_type glob -i "/home/user/event_video/frames/*.png" -c:v libx265 -x265-params lossless=1 /home/user/event_video/video.mp4`